In [2]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import os 

In [3]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [4]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

lending_data_to_load = Path("Resources/lending_data.csv")
lending_data_df = pd.read_csv(lending_data_to_load)

# Review the DataFrame
lending_data_df.head(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [5]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
X = lending_data_df.drop(columns=['loan_status'])

In [6]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [7]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [8]:
# Check the balance of our target values
loan_status_value_counts = y.value_counts()
loan_status_value_counts

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [9]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Assign a random_state of 1 to the function
logistic_regression_model = LogisticRegression(random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [10]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
loan_status_model = logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [11]:
# Make a prediction using the testing data

training_predictions = loan_status_model.predict(X_train)

testing_predictions = logistic_regression_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [12]:
# Print the balanced_accuracy score of the model

balanced_accuracy_score = balanced_accuracy_score(y_test, testing_predictions)

print(f"Balanced Accuracy Score: {balanced_accuracy_score}")


Balanced Accuracy Score: 0.9437458910723012


In [13]:
# Generate a confusion matrix for the model

test_matrix = confusion_matrix(y_test, testing_predictions)

print(test_matrix)

[[18652    77]
 [   71   584]]


In [14]:
# Print the classification report for the model

training_report = classification_report(y_train, training_predictions)

print(training_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56307
           1       0.85      0.90      0.87      1845

    accuracy                           0.99     58152
   macro avg       0.92      0.95      0.93     58152
weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Overall accuracy of the model is very high for accounts assgined a healthy value of 0, but much less accurate for accounts assigned an risky value of 1. According to the confusion matrix 18,643 loans were correctly identified as good risks. 

The presicion of the model for risky accounts certainly leaves room for imporvment, as 85% of accounts assigned risky were correct.

If this model was deployed I would expect the results to be stable for the lender, as healthy credit is assessed accurately. From the point of the lender money is left on the table as the number of applicants who did have healthy credit but were incorrectl assigned a risk status is 15% of all risk status. This is of course problematic for lendees who would be a good credit risk but will not receive credit as well. 

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [15]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

y = lending_data_df['loan_status']
X = lending_data_df.drop(columns=['loan_status'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y)


# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler_model = RandomOverSampler(random_state=1)


# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = random_oversampler_model.fit_resample(X_train, y_train)



In [16]:
# Count the distinct values of the resampled labels data

resampled_y_value_counts = pd.Series(y_resampled).value_counts()


resampled_y_value_counts


0    56239
1    56239
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [17]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resampled_logistic_regression_model = LogisticRegression(random_state=1)


# Fit the model using the resampled training data
resampled_loan_status_model = resampled_logistic_regression_model.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
resampled_training_predictions = resampled_loan_status_model.predict(X_train)
resampled_testing_predictions = resampled_logistic_regression_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [18]:
#print the balance accuracy score for both training and testing data

from sklearn.metrics import balanced_accuracy_score

# Calculate and print the balanced accuracy score for training predictions
balanced_accuracy_train = balanced_accuracy_score(y_train, resampled_training_predictions)
print(f"Balanced Accuracy (Training): {balanced_accuracy_train}")

# Calculate and print the balanced accuracy score for testing predictions
balanced_accuracy_test = balanced_accuracy_score(y_test, resampled_testing_predictions)
print(f"Balanced Accuracy (Testing): {balanced_accuracy_test}")





Balanced Accuracy (Training): 0.9937127415667844
Balanced Accuracy (Testing): 0.9973932010427196


In [19]:
# Generate a confusion matrix for the model

test_matrix = confusion_matrix(y_test, resampled_testing_predictions)

print(test_matrix)

[[18699    98]
 [    0   587]]


In [20]:
# Print the classification report for the model
training_report = classification_report(y_train, resampled_training_predictions)

print(training_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56239
           1       0.85      0.99      0.92      1913

    accuracy                           0.99     58152
   macro avg       0.93      0.99      0.96     58152
weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** When classifying good credit risks the model is still incredibly accuracte. When classifying bad risks, the precision of the model remained the same, but the recall and f1 score increased, so the number of false negatives decreased tremendously. 

The number of false negatives: indicated a good credit risk was actually classsified as a bad credit risk, was the main weakness of the first train/test of the model.